In [3]:
import requests
import pandas as pd
import time
import datetime
import pytz  

json_datas = []

def climate(local,start,start_time,end,end_time):
    API_KEY = "CJg6YBp/KNrl7YueJ48EwyvDpgka/lL0pUeuRvW84Aw+bDjTm0W1HVZSbhxk6mE4kU+jgLex72Up7mofH9hMUQ=="
    URL = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

    params = {
      'serviceKey' : API_KEY,
      'pageNo' : '1', 
      'numOfRows' : '900', 
      'dataType' : 'json',
      'dataCd' : 'ASOS', 
      'dateCd' : 'HR',
      'startDt' : start, 
      'startHh' : start_time, 
      'endDt' : end, 
      'endHh' : end_time, 
      'stnIds' : local
    }

    response = requests.get(URL, params=params)
    datas = response.json()

    for a in datas['response']['body']['items']['item']:
        stnnm = a['stnNm']  # 지점명
        tm = a['tm']  # 일시
        ws = a['ws']  # 풍속
        wd = a['wd']  # 풍향
        hm = a['hm']  # 습도
        ta = a['ta']  # 기온


        # json 형태로 저장
        json_data = {
        'tm' : tm,
        'stnNm' : stnnm,
        'stnId': local,
        'ta' : ta,
        'ws' : ws,
        'wd' : wd,
        'hm' : hm
        }
        
        json_datas.append(json_data)

    return json_datas


# 관측지점 뽑아오기
climate_point_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/mongo_db,관측지점,위경도(0701.ver).csv')
climate_point = list(climate_point_df['지점번호'])


# 현재 날짜 - 1일
def get_seoul_date():
    utc_now = pytz.utc.localize(datetime.datetime.utcnow())
    kst_now = utc_now.astimezone(pytz.timezone("Asia/Seoul")) + datetime.timedelta(days = -1)
    d = kst_now.strftime("%Y%m%d")  # 현재날짜 -1 (전날)
    # t = kst_now.strftime("%H:%M:%S")
    
    return d


date = get_seoul_date()

for a in climate_point:
    climate(a, date, '00', date, '23' )

In [4]:
json_datas

[{'hm': '94',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.4',
  'tm': '2022-07-26 00:00',
  'wd': '200',
  'ws': '0.7'},
 {'hm': '93',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.4',
  'tm': '2022-07-26 01:00',
  'wd': '230',
  'ws': '0.7'},
 {'hm': '93',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.5',
  'tm': '2022-07-26 02:00',
  'wd': '320',
  'ws': '0.7'},
 {'hm': '94',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.3',
  'tm': '2022-07-26 03:00',
  'wd': '290',
  'ws': '0.9'},
 {'hm': '93',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.3',
  'tm': '2022-07-26 04:00',
  'wd': '0',
  'ws': '0.0'},
 {'hm': '93',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.1',
  'tm': '2022-07-26 05:00',
  'wd': '290',
  'ws': '1.1'},
 {'hm': '94',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.0',
  'tm': '2022-07-26 06:00',
  'wd': '270',
  'ws': '0.7'},
 {'hm': '95',
  'stnId': 90,
  'stnNm': '속초',
  'ta': '22.2',
  'tm': '2022-07-26 07:00',
  'wd': '0',
  'ws': '0.4'},
 {'hm': '94',
  'stnId': 90,
  'stnN

NaN값 처리 -> None로 변경

In [8]:
df = pd.DataFrame(json_datas)
df[df['ws'] == '']

,tm,stnNm,stnId,ta,ws,wd,hm
1394,2022-07-26 02:00,금산,238,22.1,,,98


In [ ]:
json_datas[0]['hm']

'95'

In [9]:
# nan 데이터 None 으로 변경해주기 ->변경 안해주면 에러뜸
a_lst = []

for a in json_datas:

  if str(a['hm']) == '':
    a['hm'] = None

  if str(a['ta']) == '':
    a['ta'] = None

  if str(a['wd']) == '':
    a['wd'] = None

  if str(a['ws']) == '':
    a['ws'] = None

  a_lst.append(a)

json_datas = a_lst

In [6]:
!pip install mysqlclient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.6 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.1-cp37-cp37m-linux_x86_64.whl size=99979 sha256=8569c0a0fe61b4cf9df9a1716014e3070646544f03b32ed46ee38cd54330bdc1
  Stored in directory: /root/.cache/pip/wheels/95/2d/67/2cb3f82e435fc8e055cb2761a15a0812bf086068f6fb835462
Successfully built mysqlclient


In [10]:
import MySQLdb
from sqlalchemy import *

HOST = "ec2-35-77-169-139.ap-northeast-1.compute.amazonaws.com"
DB_USER   = "hsy"
DB_PASSWD = "1111"
DB_NAME = "wildfire"
conn = f"mysql://{DB_USER}:{DB_PASSWD}@{HOST}/{DB_NAME}?charset=utf8"

engine = create_engine(conn, encoding='utf-8')

connection = engine.connect()
metadata = MetaData()
table = Table('weather_info', metadata, autoload=True, autoload_with=engine)
query = insert(table)
result_proxy = connection.execute(query, json_datas)
result_proxy.close()